In [2]:
PMT2AFE = {
    0: 6,
    1: 14,
    2: 20,
    3: 2,
    4: 8,
    5: 10,
    6: 12,
    7: 16,
    8: 18,
    9: 22,
    10: 0,
    11: 4,
    12: 7,
    13: 11,
    14: 13,
    15: 17,
    16: 19,
    17: 23,
    18: 1,
    19: 5,
    20: 3,
    21: 9,
    22: 15,
    23: 21
}

AFE2PMT = {}

for key, val in PMT2AFE.items():
    AFE2PMT[val] = key

In [3]:
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
from uncertainties import unumpy
from uncertainties import ufloat
import healpy as hp
from scipy.interpolate import interp1d
import sys
from iminuit import Minuit
sys.path.insert(0, '/HDD/backuped/Promotion_data/Postdoc/241002_mDOM_efficiency_report')

from latexify import Latexify
%matplotlib widget


In [4]:
class ModuleScanData:
    """Manages spatial detection efficiency data using HEALPix mapping.
    
    Maps PMT detection efficiency measurements onto a spherical grid using
    HEALPix pixelization for uniform angular sampling.
    """
    def __init__(self, hp_nside):
        self._hp_nside = hp_nside
        self._make_hp_arrays()
        self._detection_maps = {}
        self._error_maps = {}
        self.wavelengths = []
        
    def _make_hp_arrays(self):
        self._npix = hp.nside2npix(self._hp_nside)
        pixels = np.arange(self._npix)
        hp_theta, hp_phi = hp.pix2ang(self._hp_nside, pixels)
        self.hp_theta = np.degrees(hp_theta)
        self.hp_phi = np.degrees(hp_phi)      

    def _create_map(self, theta:np.ndarray, phi:np.ndarray, detected_fraction:np.ndarray, detected_fraction_error:np.ndarray):
        map_data = np.zeros(self._npix)+np.nan
        error_data = np.zeros(self._npix)+np.nan
        for t, p, d, e in zip(theta, phi, detected_fraction, detected_fraction_error):
            idx = np.argmin(np.abs(self.hp_theta - t)+np.abs(self.hp_phi - p))
            map_data[idx] = d
            error_data[idx] = e
        return map_data, error_data

    def append_data(self, PMT:int, wavelength:float, theta:np.ndarray, phi:np.ndarray, detected_fraction:np.ndarray, detected_fraction_error:np.ndarray):
        assert (PMT, wavelength) not in self._detection_maps, "Data already exists"
        self._detection_maps[(PMT, wavelength)], self._error_maps[(PMT, wavelength)] = self._create_map(theta, phi, detected_fraction, detected_fraction_error)
        self.wavelengths.append(wavelength)
        
    def get_map(self, PMT:int, wavelength:float):
        return self._detection_maps[(PMT, wavelength)], self._error_maps[(PMT, wavelength)]

    def get_mDOM_map(self, wavelength:float):
        total = np.zeros(self.hp_theta.size)
        for PMT in range(12):
            total += self._detection_maps[(PMT, wavelength)][0]
        return total

    def get_wavelengths(self):
        return np.unique(self.wavelengths)
    
    
def load_measurement_data(fname:str, nside:int):
    theta, PMT, phi, wavelength, _, mu, mu_err, _, _, Nphot, Nphot_err, _, _ = np.loadtxt(fname, unpack=True)
    ufloat_mu = [ufloat(m, e) for m, e in zip(mu, mu_err)]
    ufloat_Nphot = [ufloat(m+1e-9, e+1e-9) for m, e in zip(Nphot, Nphot_err)]
    detected_fraction_ufloat = [m/n for m, n in zip(ufloat_mu, ufloat_Nphot)]
    detected_fraction = np.array([m.n for m in detected_fraction_ufloat])
    detected_fraction_error = np.array([m.s for m in detected_fraction_ufloat])
    scan_data = ModuleScanData(nside)
    for unique_wavelength in np.unique(wavelength):
        if unique_wavelength == 400:
            continue
        print(unique_wavelength)
        mask_w = wavelength == unique_wavelength
        for i in range(24):
            mask = np.logical_and(PMT == i, mask_w)
            scan_data.append_data(AFE2PMT[i], unique_wavelength, theta[mask], phi[mask], detected_fraction[mask], detected_fraction_error[mask])
    return scan_data

def load_simulation_data(fname:str, nside:int, N:float, scan_data=None):
    phi, theta, wavelength, tilt, *detected_fraction = np.loadtxt(fname, unpack=True)
    detected_fraction = np.array(detected_fraction)[:24]/N
    if scan_data == None:
        scan_data = ModuleScanData(nside)
    for unique_wavelength in np.unique(wavelength):
        mask = wavelength == unique_wavelength
        for i in range(24):
            scan_data.append_data(i, unique_wavelength, theta[mask], phi[mask], detected_fraction[i][mask], detected_fraction[i][mask])
    return scan_data

In [5]:
NSIDE = 16
measurement = load_measurement_data("pre_analysis/output_data/241014_summary_data_with_trigger_loss.dat", NSIDE)

480.0
500.0
520.0
540.0
560.0
580.0
600.0
620.0
640.0


In [6]:
geant_data = load_simulation_data("tilts/final_tilt_pTheta_wavelength_480.dat", NSIDE, 1000000)
geant_data = load_simulation_data("tilts/final_tilt_pTheta_wavelength_500.dat", NSIDE, 1000000, geant_data)

# Final efficiency fits

In [77]:
PMTLIM = 20
WVL = 5
class FitMaps:
    def __init__(self, geant_maps, measurement_maps, wavelength, factor):
        self.g4maps = geant_maps
        self.mmaps = measurement_maps
        self.masks = []
        self.wavelength = wavelength

        for pmt in range(PMTLIM):
            mes_map = self.mmaps.get_map(pmt, self.wavelength)[0]
            mask = mes_map>np.nanmax(mes_map)
            self.masks.append(mask)

    def ls_all(self, parameters):
        scaling = parameters[:PMTLIM]
        background = np.zeros(PMTLIM)#parameters[PMTLIM:]
        total = 0
        self.nr_measurements = 0
        for pmt in range(PMTLIM):
            mdl = self.scale(self.g4maps.get_map(pmt, self.wavelength)[0], scaling[pmt], 0)
            total+=self.least_squares(self.masks[pmt], mdl, *self.mmaps.get_map(pmt, self.wavelength))
            
        return total
    def least_squares(self, mask, model, mes_map, error_map ):
        ls = (mes_map[mask]-model[mask])**2/error_map[mask]**2
        self.nr_measurements += ls.size-np.sum(np.isnan(ls))
        return np.nansum(ls)
            
    def scale(self, simulation_map, scale, background=0):
        return simulation_map*scale*self.QE_factor+background
        
fctrs = np.linspace(0,0.9,20)
wavelength = np.arange(480, 660, 20)
p0 = []

for i in range(PMTLIM):
    p0.append(0.92)
chis = []
factors = []
stds = []
for WVL in wavelength:

    factor =0.01 if WVL==620 else 0 # 620 nm have two pixels that failed
    fit = FitMaps(geant_data, measurement, WVL, factor)
    m = Minuit(fit.ls_all, p0)
    m.migrad()
    chis.append(m.fval/(fit.nr_measurements))
    factors.append(np.mean(list(m.values.to_dict().values())[:PMTLIM]))
    stds.append(np.std(list(m.values.to_dict().values())[:PMTLIM]))



480 1.000091479296798
500 0.9953721684726936
520 0.9924617977545287
540 0.977344919002268
560 0.9586903891972198
580 0.9366593305440066
600 0.9052236009390802
620 0.8719727872827115
640 0.8687602589697484
